# Link Prediction

In this notebook, we demonstrate how to use GraphLab Create to construct a simple [Link Prediction](http://personal.stevens.edu/~jbao/BIA658A/Session10/liben-nowell.pdf) classifier.
A link prediction classifier is a classifier that can predict the probability of the existence (or non-existence) of a link in a social network (SN). Namely, given a social network, such as Facebook, a link prediction classifier can help to determine if a link between two users exist. Our construction is general and not limited to SN. For example, in a phone call network our classifier can predict whether two subscribers will call each other. 

There are many diverse methods to construct a link prediction classifier, such as using the SN topology, using the SN users' personal details or using posts and other published online content. 

The main goal of this notebook is to demonstrate how to use GraphLab's SFrame and SGraph to extract various SN topological features. These topological features can later be utilized to perform various prediction tasks, such as 
[link prediction](http://dl.acm.org/citation.cfm?id=2542192), and [predicting students' exam scores](http://www.academia.edu/1123106/Predicting_Student_Exams_Scores_by_Analyzing_Social_Network_Data). 

This notebook is organized as follows: First, we download a link prediction training dataset and constract a graph from it. Next, we will illustrate how GraphLab's SFrame and SGraph objects can be utilized to extract various SN topological features for each link. Finally, we will demonstrate how to use those features inside a link prediction classifier.

## Downloading the Dataset 

In this notebook, we will use dataset which was published by [BGU Social Network Research Group](http://proj.ise.bgu.ac.il/sns/datasets.html). 
We will download the [Google+ dataset](http://proj.ise.bgu.ac.il/sns/googlep.html) which consists of around 3 million links, and generate a SGraph object containing the social links.

In [1]:
import graphlab as gl

# Loading the links dataset into a SFrame object
sf_links = gl.SFrame.read_csv("https://static.turi.com/datasets/bgu_directed_network_googleplus/g_plus_pos_and_neg_links.csv.gz")

# Let's view the data
print sf_links.head(3)

# Creating SGraph object from the SFrame object
g = gl.SGraph().add_edges(sf_links, src_field='src', dst_field='dst')


[INFO] This commercial license of GraphLab Create is assigned to engr@turi.com.

[INFO] Start server at: ipc:///tmp/graphlab_server-3600 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1441218013.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Downloading https://static.turi.com/datasets/bgu_directed_network_googleplus/g_plus_pos_and_neg_links.csv.gz to /var/tmp/graphlab-bickson/3600/000000.gz
PROGRESS: Finished parsing file https://static.turi.com/datasets/bgu_directed_network_googleplus/g_plus_pos_and_neg_links.csv.gz
PROGRESS: Parsing completed. Parsed 100 lines in 1.35685 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file https://static.turi.com/datasets/bgu_directed_network_googleplus/g_plus_pos_and_neg_links.csv.gz
PROGRESS: Parsing completed. Parsed 3013792 lines in 1.60591 secs.
+-------+-------+-------+
|  src  |  dst  | class |
+-------+-------+-------+
| 16400 | 60081 |   1   |
| 55441 | 34187 

Let use the SGraph's summary function to get details on the loaded SGraph object

In [2]:
g.summary()

{'num_edges': 3013792, 'num_vertices': 211187}

Our directed graph has around 3 million edges (referred to as links) and over 200,000 vertices (referred to as users). 
Those links are roughtly divided into two. Positive links (labeled 1), and Negative links (labeled 0). The positive links are observed links in the social network, while negative links were added at random so our binary classifier could learn from the negative examples as well. Note, that if you like to use your own dataset, you will need to add negative (unobserved) edges at random for our construction to work.

## Topological Feature Extraction

Let's create an SFrame obejct with all user ids.

In [3]:
users_sf = g.get_vertices()
users_sf.rename({"__id": "id"}) 
users_sf.head(5)

id
5
7
8
10
27


The users sframe can be used for calculating several topological feature for each link in the dataset. 
We will start by calculating various simple topological features, such as the each user's in-degree (i.e. the user's number of followers) and user's out-degree (i.e. number of users the user is followling).  

To calculate each vertex toplogical features, we will first create a SFrame object that contains each user's in-friends (i.e. the users in the network that follows the user), and the user's out-friends (the users in the network which the user follows).

In [4]:
# Calculating each vertices in and out degree
out_friends_sf = sf_links.groupby("src", {"out_friends": gl.aggregate.CONCAT("dst")})
out_friends_sf.rename({"src": "id"})
in_friends_sf = sf_links.groupby("dst", {"in_friends": gl.aggregate.CONCAT("src")})
in_friends_sf.rename({"dst": "id"})


id,in_friends
211023,"[175821, 191454, 42927,19425, 20037, 84186, ..."
21855,"[117830, 202635, 7922,148051, 98174, 10749, ..."
88004,"[184267, 165121, 156061,25801, 82540, 200953, ..."
79732,"[166236, 156056, 42127,173758, 44003, 113404, ..."
63664,"[133355, 135427, 189724,52481, 74786, 148446, ..."
127950,"[108832, 124060, 191707,64471, 18708, 31194, ..."
7899,"[19009, 14262, 166769,92858, 45224, 202978, ..."
25263,"[47241, 145136, 57020,2642, 47271, 60974, ..."
130872,"[132401, 79291, 158521,197742, 6452, 48452, ..."
208837,"[122872, 14865, 23427,86661, 108812, 42584, ..."


Using the SFrame [join](https://turi.com/products/create/docs/generated/graphlab.SFrame.join.html#graphlab.SFrame.join) operation, we create a single SFrame which consists of each user's in and out friends.

In [5]:
users_sf = users_sf.join(in_friends_sf, on="id", how="outer")
users_sf = users_sf.join(out_friends_sf, on="id", how="outer")

# we replace missing values with empty lists
users_sf = users_sf.fillna('in_friends',[])
users_sf = users_sf.fillna('out_friends',[])
users_sf.head(10)

id,in_friends,out_friends
211023,"[175821, 191454, 42927,19425, 20037, 84186, ...","[178359, 191454, 114855,117085, 95731, 150495, ..."
21855,"[117830, 202635, 7922,148051, 98174, 10749, ...","[41086, 49199, 75658,55823, 67923, 199185, ..."
88004,"[184267, 165121, 156061,25801, 82540, 200953, ...","[194693, 53575, 36045,162977, 184267, 82540, ..."
79732,"[166236, 156056, 42127,173758, 44003, 113404, ...","[42127, 56032, 190023,137866, 56552, 166236, ..."
63664,"[133355, 135427, 189724,52481, 74786, 148446, ...","[189724, 148446, 74702,42791, 133355, 153655, ..."
127950,"[108832, 124060, 191707,64471, 18708, 31194, ...","[179818, 185573, 85706,207053, 27056, 44792, ..."
7899,"[19009, 14262, 166769,92858, 45224, 202978, ...","[45224, 82615, 202978,68792, 53078, 66750, ..."
25263,"[47241, 145136, 57020,2642, 47271, 60974, ...","[91704, 23796, 71943,32645, 189962, 109815, ..."
130872,"[132401, 79291, 158521,197742, 6452, 48452, ...","[108790, 43254, 114610,79291, 56216, 104160, ..."
208837,"[122872, 14865, 23427,86661, 108812, 42584, ...","[193657, 203867, 177456,107268, 12804, 161402, ..."


Using the created SFrame with each user in-friends and out-friends, we calculate several simple topological features for each user,  such as each user's in-degree (i.e. number of followers) and each user's out-degree (i.e. number of users the user is followling). 

In [6]:
#out_degree - number of users each vertex is following
users_sf['out_degree'] = users_sf["out_friends"].apply(lambda l: len(l) )

#in_degree - number of users following each vertex
users_sf['in_degree'] = users_sf["in_friends"].apply(lambda l: len(l) )

#all_degree - number of uniuqe users that following or are followed by each user
users_sf['all_friends'] = users_sf[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) | set(r['out_friends'])))
users_sf['all_degree'] = users_sf["all_friends"].apply(lambda l: len(l) )

#bi_degree - number of uniuqe users that are both following and followed by each user
users_sf['bi_friends'] = users_sf[['in_friends', 'out_friends']].apply(lambda r: list(set(r['in_friends']) & set(r['out_friends'])))
users_sf['bi_degree'] = users_sf["bi_friends"].apply(lambda l: len(l) )

users_sf.head(10)

id,in_friends,out_friends,out_degree,in_degree,all_friends
211023,"[175821, 191454, 42927,19425, 20037, 84186, ...","[178359, 191454, 114855,117085, 95731, 150495, ...",8,7,"[19425.0, 39588.0,20037.0, 84934.0, ..."
21855,"[117830, 202635, 7922,148051, 98174, 10749, ...","[41086, 49199, 75658,55823, 67923, 199185, ...",8,7,"[206059.0, 117830.0,5607.0, 75658.0, ..."
88004,"[184267, 165121, 156061,25801, 82540, 200953, ...","[194693, 53575, 36045,162977, 184267, 82540, ...",10,11,"[165121.0, 53575.0,194693.0, 9350.0, ..."
79732,"[166236, 156056, 42127,173758, 44003, 113404, ...","[42127, 56032, 190023,137866, 56552, 166236, ...",9,9,"[187648.0, 56032.0,44003.0, 57284.0, ..."
63664,"[133355, 135427, 189724,52481, 74786, 148446, ...","[189724, 148446, 74702,42791, 133355, 153655, ...",11,13,"[52481.0, 135427.0,133516.0, 74702.0, ..."
127950,"[108832, 124060, 191707,64471, 18708, 31194, ...","[179818, 185573, 85706,207053, 27056, 44792, ...",19,29,"[186883.0, 71174.0,138631.0, 98699.0, ..."
7899,"[19009, 14262, 166769,92858, 45224, 202978, ...","[45224, 82615, 202978,68792, 53078, 66750, ...",11,10,"[19009.0, 202978.0,199941.0, 149921.0, ..."
25263,"[47241, 145136, 57020,2642, 47271, 60974, ...","[91704, 23796, 71943,32645, 189962, 109815, ...",7,9,"[71943.0, 32645.0,47271.0, 47241.0, ..."
130872,"[132401, 79291, 158521,197742, 6452, 48452, ...","[108790, 43254, 114610,79291, 56216, 104160, ...",8,9,"[104160.0, 188507.0,48452.0, 43254.0, ..."
208837,"[122872, 14865, 23427,86661, 108812, 42584, ...","[193657, 203867, 177456,107268, 12804, 161402, ...",7,17,"[23427.0, 107268.0,86661.0, 200330.0, ..."


Now, we  have several degree feautres for each user. Lets utilize these users' features and create features for each link in our positive and negative links dataset.
Namely, for each link in the data conists of two users <i>u</i> and <i>v</i>, we will create SFrame with each user's degree features.

Note: runining can take several minutes.

In [7]:

sf_links = sf_links.join(users_sf, on={"src": "id"}, how="right")
sf_links.rename({"in_friends": "src_in_friends", "out_friends": "src_out_friends",
           "all_friends": "src_all_friends", "all_degree": "src_all_degree",
           "bi_friends": "src_bi_friends", "bi_degree": "src_bi_degree",
           "in_degree": "src_in_degree", "out_degree": "src_out_degree"
           })

sf_links = sf_links.join(users_sf, on={"dst": "id"}, how="right")
sf_links.rename({"in_friends": "dst_in_friends", "out_friends": "dst_out_friends",
           "all_friends": "dst_all_friends", "all_degree": "dst_all_degree",
           "bi_friends": "dst_bi_friends", "bi_degree": "dst_bi_degree",
           "in_degree": "dst_in_degree", "out_degree": "dst_out_degree"})

sf_links.head(10)


src,dst,class,src_in_friends,src_out_friends,src_out_degree,src_in_degree
16400,60081,1,"[398, 164476, 158289,25877, 45051, 103519, ...","[60081, 72458, 55255,23667, 45842, 510, ...",187,60
55441,34187,1,"[47069, 203601, 202891,74718, 40220, 95893, ...","[34187, 169679, 20260,2828, 166652, 104248, ...",12,9
59472,67007,1,"[153627, 170956, 59785,107075, 42942, 119730, ...","[67007, 68849, 13940,66935, 207534, 202531, ...",101,102
79102,174248,1,"[17824, 86736, 40718,80705, 183565, 206244, ...","[174248, 58527, 94279,82240, 150362, 115046, ...",378,125
135415,63941,1,"[25504, 159023, 79854,85472, 899, 180460, ...","[63941, 123425, 33739,6473, 2925, 154425, ...",86,54
21650,154098,1,"[105155, 20292, 166059,71126, 510, 187171, ...","[154098, 171451, 145995,10888, 164476, 122311, ...",571,221
140358,152017,1,"[26754, 196070, 181832,89802, 127264, 150731, ...","[152017, 21469, 181065,45442, 186338, 55170, ...",142,169
131272,19556,1,"[177562, 119155, 4050,80338, 84221, 113372, ...","[19556, 142061, 14344,98752, 191884, 31489, ...",101,72
49627,29754,1,"[54190, 29754, 84452,126486, 208641, 135083, ...","[29754, 40361, 126486,209533, 29482, 105542, ...",9,11
44402,82787,1,"[169958, 17147, 171733,138035, 118118, 28173, ...","[82787, 117486, 196181,107067, 84349, 172619, ...",307,533


Beside adding each link's users <i> u </i> and <i> v </i> degree features, to create a decent link prediction classifier, we also need to add features based on the strength of connection between the users.
In this notebook, we will add for each link three simple type of features:
- <i>Common-Friends Features</i> - the number of friends both <i>u</i> and <i>v</i> have in common. 
- <i>Total-Friends Features</i> - the number of friends both <i>u</i> and <i>v</i> have in together.
- <i>Jaccard coefficient</i>- the number of Common-Friends divided by the Number of Total-Friends.

Lets define the each feature type function:


In [8]:
def common_friends(u, v, u_friends, v_friends):
    u_friends = set(u_friends)
    if v in u_friends:
            u_friends.remove(v)

    v_friends = set(v_friends)
    if u in v_friends:
        v_friends.remove(u)
    return len(u_friends & v_friends)

def total_friends(u, v, u_friends, v_friends):
    u_friends = set(u_friends)
    if v in u_friends:
        u_friends.remove(v)

    v_friends = set(v_friends)
    if u in v_friends:
        v_friends.remove(u)

    return len(u_friends | v_friends)

def jacc_coef(u,v, u_friends, v_friends):
    t = total_friends(u,v,u_friends,v_friends)
    if  t == 0:
        return 0
    return common_friends(u,v,u_friends, v_friends)/ float(t)


Using these three features type we created 12 new features (4 feature for each feature type) that are based on direction of the friendship between 
<i>u</i> and <i>v</i> and their friends. 

Please note that The formal mathematical defintion of the feature presented throught this section can be found in [Fire et al. 2014](http://dl.acm.org/citation.cfm?id=2542192).

The following code block may take a few minutes.

In [9]:
sf_links['common_friends'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['common_bi_friends'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['common_in_friends'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['common_out_friends'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: common_friends(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))

sf_links['total_friends'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['total_bi_friends'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['total_in_friends'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['total_out_friends'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: total_friends(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))


sf_links['jacc_coef'] = sf_links[['src','dst', 'src_all_friends', 'dst_all_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_all_friends'], r['dst_all_friends']))
sf_links['bi_jacc_coef'] = sf_links[['src','dst', 'src_bi_friends', 'dst_bi_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_bi_friends'], r['dst_bi_friends']))
sf_links['in_jacc_coef'] = sf_links[['src','dst', 'src_in_friends', 'dst_in_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_in_friends'], r['dst_in_friends']))
sf_links['out_jacc_coef'] = sf_links[['src','dst', 'src_out_friends', 'dst_out_friends']].apply(lambda r: jacc_coef(r['src'], r['dst'],r['src_out_friends'], r['dst_out_friends']))
sf_links.head(10)

src,dst,class,src_in_friends,src_out_friends,src_out_degree,src_in_degree
16400,60081,1,"[398, 164476, 158289,25877, 45051, 103519, ...","[60081, 72458, 55255,23667, 45842, 510, ...",187,60
55441,34187,1,"[47069, 203601, 202891,74718, 40220, 95893, ...","[34187, 169679, 20260,2828, 166652, 104248, ...",12,9
59472,67007,1,"[153627, 170956, 59785,107075, 42942, 119730, ...","[67007, 68849, 13940,66935, 207534, 202531, ...",101,102
79102,174248,1,"[17824, 86736, 40718,80705, 183565, 206244, ...","[174248, 58527, 94279,82240, 150362, 115046, ...",378,125
135415,63941,1,"[25504, 159023, 79854,85472, 899, 180460, ...","[63941, 123425, 33739,6473, 2925, 154425, ...",86,54
21650,154098,1,"[105155, 20292, 166059,71126, 510, 187171, ...","[154098, 171451, 145995,10888, 164476, 122311, ...",571,221
140358,152017,1,"[26754, 196070, 181832,89802, 127264, 150731, ...","[152017, 21469, 181065,45442, 186338, 55170, ...",142,169
131272,19556,1,"[177562, 119155, 4050,80338, 84221, 113372, ...","[19556, 142061, 14344,98752, 191884, 31489, ...",101,72
49627,29754,1,"[54190, 29754, 84452,126486, 208641, 135083, ...","[29754, 40361, 126486,209533, 29482, 105542, ...",9,11
44402,82787,1,"[169958, 17147, 171733,138035, 118118, 28173, ...","[82787, 117486, 196181,107067, 84349, 172619, ...",307,533


Similar to the above method, we can also extract for each link, such as each user [PageRank](https://turi.com/products/create/docs/generated/graphlab.pagerank.create.html) and [neighborhood subgraph](https://turi.com/products/create/docs/generated/graphlab.SGraph.get_neighborhood.html?highlight=neighborhood#graphlab.SGraph.get_neighborhood) size. We let the reader to try to add these features (and  maybe some additional features) by themselves. 

Let us move to the next section, in which we explain how the extracted links' features can be utilized to create a link prediction classifier.

## Constructing a Link Prediction Classifier

In order to create a link prediction classifier using our constructed links dataset (sf), lets first randomly split our dataset into training that contains 20% of the dataset, and testing datasets that contains 80% of the dataset.

In [10]:
train, test = sf_links.random_split(0.2)

We now can use GraphLab Create's [classfication toolkit](https://turi.com/products/create/docs/graphlab.toolkits.classifier.html#creating-a-classifier) and create and evaluate a link prediction classifier based only on <i>u</i> and <i>v</i> degree features:


In [11]:
degree_features_list = [c for c in train.column_names() if "degree" in c]
print "Degree Features %s" % degree_features_list
cls = gl.classifier.create(train,features=degree_features_list, target="class")
results = cls.evaluate(test)
print results

Degree Features ['src_out_degree', 'src_in_degree', 'src_all_degree', 'src_bi_degree', 'dst_out_degree', 'dst_in_degree', 'dst_all_degree', 'dst_bi_degree']
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, SVMClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 572642
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 8
PROGRESS: Number of unpacked features : 8
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
P

We get pretty good accuracy of 0.88. Let us add also the link based features and use the Boosted-Trees classifier to create and evaluate a link predicdiction classifier.

In [12]:
link_features_list = ['common_friends', 'common_in_friends', 'common_out_friends', 'common_bi_friends', 'total_friends', 'total_in_friends', 'total_out_friends', 'total_bi_friends',
'jacc_coef', 'bi_jacc_coef', 'in_jacc_coef', 'out_jacc_coef']
cls = gl.classifier.boosted_trees_classifier.create(train,target="class", features=degree_features_list + link_features_list)
results = cls.evaluate(test)
print results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 572759
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 20
PROGRESS: Number of unpacked features : 20
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   9.208e-01   9.185e-01        2.91s
PROGRESS:      1   9.238e-01   9.219e-01        5.34s
PROGRESS:      2   9.241e-01   9.221e-01        7.61s
PROGRESS:      3   9.334e-01   9.323e-01        9.99s
PROGRESS:      4   9.370e-01   9.349e-01       12.45s
PROGRESS:      5   9.386e-01   9.369e-01       14.79s
PROGRESS:      6   9.384e-01   9

Using the additional link features, we got considerbly better accuracy of around 0.94. We can try to further improve the accuracy by adding additional features or by increasing the size of the training dataset.



## Further Readings

Further reading materials can be found in the following links:
- [Stanford Large Network Dataset Collection (SNAP)](https://snap.stanford.edu/data/)
- [BGU Social Networks Security Research Group](http://proj.ise.bgu.ac.il/sns/)
- Liben‐Nowell, David, and Jon Kleinberg. "The link‐prediction problem for social networks." Journal of the American society for information science and technology 58.7 (2007): 1019-1031. ‏
- Al Hasan, Mohammad, and Mohammed J. Zaki. "A survey of link prediction in social networks." Social network data analytics. Springer US, 2011. 243-275.‏
- Fire, Michael, et al. "Link prediction in social networks using computationally efficient topological features." Privacy, Security, Risk and Trust (PASSAT) and 2011 IEEE Third Inernational Conference on Social Computing (SocialCom), 2011 IEEE Third International Conference on. IEEE, 2011.‏